In [ ]:
pip install scikit-learn

In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split

In [ ]:
path = 'd:\\Users\\Patri\\Documents\\1.TCC\\Notebooks\\WebScrapAcordaosPDF\\NLTK'
os.chdir(path)

In [ ]:
%%time

# Ler Arquivo gravado com todo o tratamento dado até aqui
# dfcsv = pd.read_csv('Acordaos2020booleado.csv')
dfcsv = pd.read_csv('Acordaos2020tx.csv')

In [ ]:
sum_num = 137
sum_str = 'S'+str(sum_num).zfill(3)     #S999
sum_RE = sum_str + 'RE'
sum_VO = sum_str + 'VO'

dfcsv[[sum_RE, sum_VO]].value_counts() 

In [ ]:
df_RE0 = dfcsv.query(sum_RE + ' == 0')   # Selecionar apenas acórdãos cujos relatórios não mencionaram a súmula 
df_RE0[[sum_RE, sum_VO]].value_counts()

In [ ]:
df, validation = train_test_split(df_RE0, train_size = 2000)

train, test = train_test_split(df)

print(len (df), len (train), len (test), len (validation))

In [ ]:
df

In [ ]:
df = df[['Decisao', 'Relatorio', sum_VO]]
df = df.reset_index(drop = True)
dataset_2020 = df
dataset_2020

In [ ]:
dataset_nltk = dataset_2020

In [ ]:
path = 'd:\\Users\\Patri\\Documents\\1.TCC\\Notebooks\\WebScrapAcordaosPDF\\NLTK'
dataset_precedentes = pd.read_csv(path+'\\dataset_precedentes_' + str(sum_num) + '.csv')
dataset_precedentes

In [ ]:
dataset_nltk = pd.concat ([dataset_2020, dataset_precedentes], ignore_index=True)
dataset_nltk

In [ ]:
train, test = train_test_split(dataset_nltk)

# model generation
X_train = train['Relatorio']
y_train = train[sum_VO]
X_test = test['Relatorio']
y_test = test[sum_VO]

In [ ]:
y_train.value_counts()

In [ ]:
import nltk
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')
stopwords

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

In [ ]:
pipe = Pipeline([
    ('TFIDF', TfidfVectorizer(stop_words=stopwords)),
    ('classification', MultinomialNB())
    ])
pipe.fit(X_train,y_train)
predicted = pipe.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Precision:",metrics.precision_score(y_test, predicted))
print("Recall:",metrics.recall_score(y_test, predicted))
conf_mnnb = confusion_matrix(y_test, predicted)
conf_mnnb

In [ ]:
pipe = Pipeline([
    ('TFIDF', TfidfVectorizer(stop_words=stopwords)),
    ('classification', LogisticRegression())
    ])
pipe.fit(X_train,y_train)
predicted = pipe.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Precision:",metrics.precision_score(y_test, predicted))
print("Recall:",metrics.recall_score(y_test, predicted))
conf_mnnb = confusion_matrix(y_test, predicted)
conf_mnnb

In [ ]:
pipe = Pipeline([
    ('TFIDF', TfidfVectorizer(stop_words=stopwords)),
    ('classification', RandomForestClassifier())
    ])
pipe.fit(X_train,y_train)
predicted = pipe.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Precision:",metrics.precision_score(y_test, predicted))
print("Recall:",metrics.recall_score(y_test, predicted))
conf_mnnb = confusion_matrix(y_test, predicted)
conf_mnnb

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.tokenize import RegexpTokenizer
from nltk import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import regex as re
import requests
from sklearn.svm import SVC

In [ ]:
df = dataset_nltk
df

In [ ]:
from nltk.corpus import stopwords

# Recebe um texto ou series e retorna os tokens
def tokeniza(texto):
    if type(texto) == "Series":
        acordao_text = texto.to_string()
    else:
        acordao_text = str(texto)
    acordao_text = acordao_text.lower() # coloca tudo em minúsculo
    acordao_text = re.sub('(º)', ' ', acordao_text) # remove 'º' substituindo por espaço
    acordao_words = RegexpTokenizer(r'\w+') # remove pontuação ao selecionar apenas palavras
    acordao_tokens = acordao_words.tokenize(acordao_text) # tokeniza (separa as palavras)

    # Extrair Stopwords em Português
    pt_stops = set(stopwords.words('portuguese'))
    pt_stops.update(['fls', 'cuida', 'ª', 'n', 'fl'])
    words = (acordao_tokens)
    lista_words = [word for word in words if word not in pt_stops] # remove stopwords
    if not len(lista_words): lista_words = []
    return lista_words 

# Remove números de um texto
def remove_numeros(texto):
    output = ' '.join(c for c in texto if not c.isdigit())
    return output

def lemma(x):
    lemmatizer = WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(word) for word in x])

In [ ]:
df['tokens']=df['Relatorio'].map(tokeniza)
df['lemma'] = df['tokens'].map(lemma)

In [ ]:
df['lemma'].values[0]

In [ ]:
df['tokens'].values[0]

In [ ]:
ptstopwords = ['º', 'ª', 'a', 'ainda', 'ao', 'aos', 'apenas', 'apos', 'após', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aquilo', 'art', 'artigo', 'arts', 'as', 'assim', 'assunto', 'até', 'brasil', 'cabe', 'caracteriza', 'caso', 'com', 'como', 'conforme', 'correspondente', 'da', 'das', 'data', 'de', 'dela', 'delas', 'dele', 'deles', 'depois', 'desde', 'direito', 'do', 'dos', 'e', 'ela', 'elas', 'ele', 'eles', 'em', 'entre', 'era', 'eram', 'essa', 'essas', 'esse', 'esses', 'esta', 'estamos', 'estas', 'estava', 'estavam', 'este', 'esteja', 'estejam', 'estejamos', 'estes', 'esteve', 'estive', 'estivemos', 'estiver', 'estivera', 'estiveram', 'estiverem', 'estivermos', 'estivesse', 'estivessem', 'estivéramos', 'estivéssemos', 'estou', 'está', 'estávamos', 'estão', 'eu', 'falar', 'fl', 'fls', 'foi', 'fomos', 'for', 'fora', 'foram', 'forem', 'formos', 'fosse', 'fossem', 'fui', 'fôramos', 'fôssemos', 'haja', 'hajam', 'hajamos', 'havemos', 'hei', 'houve', 'houvemos', 'houver', 'houvera', 'houveram', 'houverei', 'houverem', 'houveremos', 'houveria', 'houveriam', 'houvermos', 'houverá', 'houverão', 'houveríamos', 'houvesse', 'houvessem', 'houvéramos', 'houvéssemos', 'há', 'hão', 'ii', 'in', 'inciso', 'isso', 'isto', 'já', 'lhe', 'lhes', 'mais', 'mas', 'me', 'mesma', 'mesmo', 'meu', 'meus', 'minha', 'minhas', 'muito', 'n', 'na', 'nao', 'nas', 'nem', 'no', 'nos', 'nossa', 'nossas', 'n', 'nosso', 'nossos', 'num', 'numa', 'nº', 'não', 'nós', 'o', 'os', 'ou', 'para', 'pela', 'pelas', 'pelo', 'pelos', 'pode', 'pois', 'por', 'qual', 'qualquer', 'quando', 'quanto', 'que', 'quem', 'se', 'seja', 'sejam', 'sejamos', 'sem', 'sendo', 'ser', 'serei', 'seremos', 'seria', 'seriam', 'será', 'serão', 'seríamos', 'seu', 'seus', 'sido', 'sobre', 'somos', 'sou', 'sua', 'suas', 'são', 'só', 'tais', 'tal', 'tambem', 'também', 'te', 'tem', 'temos', 'tendo', 'tenha', 'tenham', 'tenhamos', 'tenho', 'ter', 'terei', 'teremos', 'teria', 'teriam', 'termos', 'terá', 'terão', 'teríamos', 'teu', 'teus', 'teve', 'tinha', 'tinham', 'tive', 'tivemos', 'tiver', 'tivera', 'tiveram', 'tiverem', 'tivermos', 'tivesse', 'tivessem', 'tivéramos', 'tivéssemos', 'todos', 'trata', 'tu', 'tua', 'tuas', 'tém', 'tínhamos', 'um', 'uma', 'valor', 'valores', 'vez', 'você', 'vocês', 'vos', 'à', 'às', 'é', 'éramos']

In [ ]:
X = df['lemma']
y = df[sum_VO]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 123)

In [ ]:
pipe_mnnb = Pipeline(steps = [('tf', TfidfVectorizer()), ('mnnb', MultinomialNB())])

pgrid_mnnb = {
 'tf__max_features' : [1000, 2000, 3000],
 'tf__stop_words' : ['portuguese', None, ptstopwords],
 'tf__ngram_range' : [(1,1),(1,2)],
 'tf__use_idf' : [True, False],
 'mnnb__alpha' : [0.1, 0.5, 1]
}

gs_mnnb = GridSearchCV(pipe_mnnb,pgrid_mnnb,cv=5,n_jobs=-1)
gs_mnnb.fit(X_train, y_train)
preds_mnnb = gs_mnnb.predict(X)

print("Accuracy:",metrics.accuracy_score(y, preds_mnnb))
print("Precision:",metrics.precision_score(y, preds_mnnb))
print("Recall:",metrics.recall_score(y, preds_mnnb))
conf_mnnb = confusion_matrix(y, preds_mnnb)
print(conf_mnnb)
gs_mnnb.best_params_

In [ ]:
df['preds'] = preds_mnnb

In [ ]:
df[[sum_VO]].value_counts()

In [ ]:
df[[sum_VO,'preds']].value_counts()

In [ ]:
df[[sum_VO,'preds']].query(sum_VO + ' == 1')